In [8]:
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
print(TTS().list_models())

# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# Run TTS
# ❗ Since this model is multi-lingual voice cloning model, we must set the target speaker_wav and language
# Text to speech list of amplitude values as output
# wav = tts.tts(text="Hello world!", speaker_wav="/workspace/face_anim_project/ref_audio/male_ref_2_16.wav", language="en")
# Text to speech to a file
tts.tts_to_file(text="<happy> This forest... it’s more confusing than the maps suggested.", speaker_wav="/workspace/face_anim_project/ref_audio/male_ref_2_min.wav", language="en", file_path="/workspace/face_anim_project/male_test_happy_2min.wav")

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
 > Text splitted to sentences.
['This forest... it’s more confusing than the maps suggested.']
 > Processing time: 3.5181386470794678
 > Real-time factor: 0.7265749772225972


'/workspace/face_anim_project/male_test_happy_2min.wav'

In [ ]:
list_of_male_lines = ["This forest... it’s more confusing than the maps suggested.", "Yes, charming indeed. If only charm could guide us back on course.",
                      "","",
                     "","",
                     "","",
                     "",""]
list_of_female_lines = ["Aye, but that’s part of its charm, isn’t it? Look at the way the light dances through the leaves.", "We mustn’t let ourselves get lost in wonder when we have a task at hand. We need to find the sacred artifact.",
                       "","",
                     "","",
                     "","",
                     "",""]

In [1]:
dict_with_lines = {'SIR CEDRIC  ': {1: 'This forest... it’s more confusing than the maps suggested.',
                                    3: 'Yes, charming indeed. If only charm could guide us back on course.',
                                    5: 'Right. A task. It’s just... what if I’m not the knight they think I am? What if I can’t lead us out of here?',
                                    7: 'It’s not just a forest! It’s enchanted, filled with ancient magic. I feel like a fool wandering blindly while you—',
                                    9: 'Together... yes. But how can we trust each other if we can’t even trust ourselves?',
                                    11: "You did what you thought was right. We all face our failures—it's part of being human.",
                                    13: 'Alright then, one foot at a time it is. Which way do you think we should go?',
                                    15: 'Lead on, Lady Elara. Let us see where this path takes us.'},
                   'LADY ELARA  ': {2: 'Aye, but that’s part of its charm, isn’t it? Look at the way the light dances through the leaves.',
                                    4: 'We mustn’t let ourselves get lost in wonder when we have a task at hand. We need to find the sacred artifact.',
                                    6: 'Sir Cedric, you’ve faced dragons and armies. You’ve fought for honor. Do you really doubt yourself in a mere forest?',
                                    8: "While I what? Have the forest's secrets whispered in my ear? Cedric, this place is unpredictable for everyone. We’re in this together.",
                                    10: 'You’re not alone in this. I carry my own burdens as well. Remember when I failed my last quest? I felt like a shadow of my former self.',
                                    12: 'Exactly. And it’s how we rise from those failures that truly defines us. Let’s take each step together, one foot at a time.',
                                    14: 'Let’s follow the sound of the water. Water often leads to life—and perhaps to our artifact.'}}


In [18]:
def inference(dict_of_replics, output_folder):
        dict_with_audio = {}
        for key, value in dict_of_replics.items():
            #get the ref source audio path by the name
            print(key)
            for ind, line in value.items():
                # dict_with_audio[key][ind] = self.generate(path_to_ref_audio = ref_audio, 
                #                                           source_transcript = ref_transcript,
                #                                           output_transcript = line)
                key_to_write = key.replace(' ', '_')
                key_to_write = key_to_write.replace('__','')
                # print(key_to_write)
                if key == 'SIR CEDRIC  ':
                    tts.tts_to_file(text=line,
                                speaker_wav="/workspace/face_anim_project/ref_audio/male_ref_2_min.wav",
                                language="en", file_path=f"/workspace/face_anim_project/CTTS_TEST/{key_to_write}_{ind}.wav")
                elif key == 'LADY ELARA  ':
                    tts.tts_to_file(text=line,
                                speaker_wav="/workspace/face_anim_project/ref_audio/gg_2_min.wav",
                                language="en", file_path=f"/workspace/face_anim_project/CTTS_TEST/{key_to_write}_{ind}.wav")
        

In [19]:
inference(dict_with_lines, None)

SIR CEDRIC  
 > Text splitted to sentences.
['This forest... it’s more confusing than the maps suggested.']
 > Processing time: 3.7939319610595703
 > Real-time factor: 0.7293224276517255
 > Text splitted to sentences.
['Yes, charming indeed.', 'If only charm could guide us back on course.']
 > Processing time: 4.741457939147949
 > Real-time factor: 0.7343584763304414
 > Text splitted to sentences.
['Right.', 'A task.', 'It’s just... what if I’m not the knight they think I am?', 'What if I can’t lead us out of here?']
 > Processing time: 8.669091701507568
 > Real-time factor: 0.7816731222939097
 > Text splitted to sentences.
['It’s not just a forest!', 'It’s enchanted, filled with ancient magic.', 'I feel like a fool wandering blindly while you—']
 > Processing time: 8.671023845672607
 > Real-time factor: 0.7573201557334155
 > Text splitted to sentences.
['Together... yes.', 'But how can we trust each other if we can’t even trust ourselves?']
 > Processing time: 5.512273073196411
 > Rea

In [4]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf
import os

device = "cuda:0" if torch.cuda.is_available() else "cpu"

def single_inference_parler(model, tokenizer, prompt, description, output_folder_path, output_file_name):
    # if model_size == 'mini':
    #     model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-v1").to(device)
    #     tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-v1")
    # elif model_size=='large':
    #     model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-large-v1").to(device)
    #     tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-large-v1")

    input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
    prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
    audio_arr = generation.cpu().numpy().squeeze()
    sf.write(os.path.join(output_folder_path, output_file_name), audio_arr, model.config.sampling_rate)

def general_inference(dict_of_replics, output_folder_path = '/workspace/face_anim_project/parler_tts_test_2/'):
    model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-large-v1").to(device)
    tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-large-v1")

    dict_with_audio = {}
    for key, value in dict_of_replics.items():
        print(key)
        for ind, line in value.items():
            key_to_write = key.replace(' ', '_')
            key_to_write = key_to_write.replace('__','')
            
            model_size = 'large'
            prompt = line
            # print(key_to_write)
            if key == 'SIR CEDRIC  ':
                output_file_name = f'Jon_{key_to_write}_{ind}.wav'
                description = 'Jon speaks slightly animatedly and slightly slowly in delivery, with a very close recording that has no background noise.'
                single_inference_parler(model, tokenizer, prompt, description, output_folder_path, output_file_name)
                
            elif key == 'LADY ELARA  ':
                output_file_name = f'Lea_{key_to_write}_{ind}.wav'
                description = 'Lea speaks slightly animatedly and slightly slowly in delivery, with a very close recording that has no background noise.'
                single_inference_parler(model, tokenizer, prompt, description, output_folder_path, output_file_name)


In [5]:
general_inference(dict_with_lines)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.47it/s]


SIR CEDRIC  
LADY ELARA  
